In [ ]:
!pip install sec-edgar-downloader

In [ ]:
!pip install sec-parsers # base package
!pip install sec-parsers['all'] # installs all extras
!pip install sec-parsers['downloaders'] # installs downloaders extras
!pip install sec-parsers['visualizers'] # installs visualizers extras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.7 MB/s eta 0:00:00


final

In [ ]:
import pandas as pd
df=pd.read_csv("/content/sp500_companies.csv")
print(df['Symbol'].tolist())

['AAPL', 'NVDA', 'MSFT', 'GOOG', 'GOOGL', 'AMZN', 'META', 'BRK-B', 'AVGO', 'LLY', 'TSLA', 'WMT', 'JPM', 'V', 'XOM', 'UNH', 'ORCL', 'MA', 'HD', 'PG', 'JNJ', 'COST', 'ABBV', 'NFLX', 'BAC', 'KO', 'CRM', 'MRK', 'CVX', 'TMUS', 'AMD', 'PEP', 'ACN', 'LIN', 'TMO', 'MCD', 'CSCO', 'WFC', 'ADBE', 'IBM', 'BX', 'GE', 'ABT', 'DHR', 'MS', 'AXP', 'CAT', 'QCOM', 'NOW', 'PM', 'ISRG', 'VZ', 'TXN', 'DIS', 'NEE', 'AMGN', 'INTU', 'RTX', 'GS', 'UBER', 'PFE', 'SPGI', 'CMCSA', 'LOW', 'T', 'AMAT', 'BLK', 'UNP', 'PGR', 'BKNG', 'LMT', 'HON', 'SYK', 'ETN', 'TJX', 'SCHW', 'BSX', 'KKR', 'ANET', 'VRTX', 'NKE', 'MU', 'COP', 'PANW', 'CB', 'ADP', 'C', 'MDT', 'UPS', 'PLD', 'FI', 'ADI', 'DE', 'MMC', 'SBUX', 'REGN', 'GILD', 'BMY', 'HCA', 'AMT', 'SO', 'ELV', 'SHW', 'INTC', 'PLTR', 'ICE', 'MDLZ', 'BA', 'LRCX', 'CI', 'DUK', 'KLAC', 'TT', 'MCO', 'DELL', 'ZTS', 'CTAS', 'ABNB', 'WM', 'EQIX', 'GD', 'MO', 'CEG', 'PYPL', 'PH', 'CME', 'CL', 'WELL', 'CMG', 'APH', 'MSI', 'TDG', 'SNPS', 'AON', 'NOC', 'ITW', 'USB', 'CRWD', 'CVS', 'PNC',

In [ ]:
import os
import shutil
import re
import csv
from sec_edgar_downloader import Downloader
from sec_parsers import Filing
import pandas as pd
import time  # Import the time module

# Initialize SEC Edgar Downloader
edgar_downloader = Downloader("MyCompanyName", "my.email@domain.com")

# Function to download filings and extract Section 7 (MD&A)
def process_filings_and_extract_mda(tickers, start_year, end_year, delay=5):
    # Create a folder for saving Section 7 data
    output_folder = "section_7_mda_results"
    os.makedirs(output_folder, exist_ok=True)

    # Track tickers successfully processed, failed, and missing Section 7
    tickers_processed = []
    tickers_failed = []
    missing_section_7 = {}

    # List of possible titles for MD&A section
    possible_titles = [
        "Item 7. Management's Discussion and Analysis",
        "Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations",
        "Management's Discussion and Analysis",
        "MD&A",
        "Item 7",
        "Management's Analysis of Financial Performance",
        "Management's Discussion and Analysis of Financial Performance",
        "Discussion and Analysis of Financial Condition and Results of Operations",
        "Item 7",
        "Management’s Discussion and Analysis of Financial Condition and Results of Operations",
        "Item 7."
    ]

    for ticker in tickers:
        try:
            print(f"Processing filings for ticker: {ticker}")

            # Download 10-K filings for the given ticker
            edgar_downloader.get("10-K", ticker)

            # Directory containing the downloaded filings
            filings_dir = f"sec-edgar-filings/{ticker}/10-K/"

            # Initialize list for missing years
            missing_years = []

            # Process each filing in the directory
            for folder in os.listdir(filings_dir):
                folder_path = os.path.join(filings_dir, folder)
                if os.path.isdir(folder_path):
                    # Extract year from the folder name
                    year_match = re.search(r'-(\d{2})-', folder)
                    if year_match:
                        year = f"19{year_match.group(1)}" if int(year_match.group(1)) > 50 else f"20{year_match.group(1)}"
                        if not (start_year <= int(year) <= end_year):
                            print(f"Skipping filing from year {year} (outside specified range).")
                            continue
                    else:
                        print(f"Year not found in folder: {folder}")
                        continue

                    # Process each HTML or text file in the folder
                    for file in os.listdir(folder_path):
                        if file.endswith(".html") or file.endswith(".txt"):
                            file_path = os.path.join(folder_path, file)
                            try:
                                # Read the local file content
                                with open(file_path, "r", encoding="utf-8") as f:
                                    file_content = f.read()

                                # Initialize Filing object with file content
                                filing = Filing(file_content)
                                filing.parse()

                                # Search for all possible titles for Section 7 (MD&A)
                                sections = []
                                for title in possible_titles:
                                    sections.extend(filing.find_all_sections_from_title(title))

                                if not sections:
                                    print(f"No 'Item 7' or variations found in {file_path}.")
                                    missing_years.append(year)  # Track year with missing Section 7
                                    continue

                                # Save extracted Section 7 to a CSV
                                csv_file = os.path.join(output_folder, f"{ticker}_{year}_section_7_mda.csv")
                                with open(csv_file, mode='w', encoding='utf-8', newline='') as csv_out:
                                    writer = csv.writer(csv_out)
                                    writer.writerow(['Section Title', 'Content'])  # Header
                                    for section in sections:
                                        title = section.attrib.get('title', 'Unknown')
                                        content = filing.get_text_from_section(section)
                                        writer.writerow([title, content])
                                print(f"Saved Section 7 for {ticker} ({year}) to {csv_file}.")

                            except Exception as e:
                                print(f"Error processing {file_path}: {e}")

            # Add ticker to processed list
            tickers_processed.append(ticker)

            # Store missing years for the ticker
            if missing_years:
                missing_section_7[ticker] = missing_years

            # Clean up: delete the downloaded filing directory
            shutil.rmtree(filings_dir)
            print(f"Cleaned up files for {ticker}.")

            # Add delay after processing each ticker
            time.sleep(delay)  # Delay in seconds (default 5 seconds)

        except Exception as e:
            print(f"Failed to process filings for ticker {ticker}: {e}")
            tickers_failed.append(ticker)

    # Summary of processing
    print("\nProcessing Summary:")
    print(f"Tickers Processed: {tickers_processed}")
    print(f"Tickers Failed: {tickers_failed}")
    print("Missing Section 7:")
    for ticker, years in missing_section_7.items():
        print(f"{ticker}: {years}")

# Example usage
if __name__ == "__main__":
    # List of tickers to process
    tickers = df['Symbol'].tolist()[100:105]

    # Specify the range of years
    start_year = 1900
    end_year = 2024

    # Process the filings and extract Section 7
    process_filings_and_extract_mda(tickers, start_year, end_year)


Processing filings for ticker: SO


/usr/local/lib/python3.10/dist-packages/sec_parsers/parsers_helper.py:23: UserWarning: DocumentType not found in metadata. Filing type set to 10K. If this is not correct, please set the filing type manually.
  warnings.warn('DocumentType not found in metadata. Filing type set to 10K. If this is not correct, please set the filing type manually.')


Saved Section 7 for SO (2012) to section_7_mda_results/SO_2012_section_7_mda.csv.
No 'Item 7' or variations found in sec-edgar-filings/SO/10-K/0000092122-98-000058/full-submission.txt.
Saved Section 7 for SO (2011) to section_7_mda_results/SO_2011_section_7_mda.csv.
Saved Section 7 for SO (2008) to section_7_mda_results/SO_2008_section_7_mda.csv.
Saved Section 7 for SO (2009) to section_7_mda_results/SO_2009_section_7_mda.csv.
No 'Item 7' or variations found in sec-edgar-filings/SO/10-K/0000092122-99-000040/full-submission.txt.
Saved Section 7 for SO (2021) to section_7_mda_results/SO_2021_section_7_mda.csv.
No 'Item 7' or variations found in sec-edgar-filings/SO/10-K/0000092122-04-000119/full-submission.txt.
Saved Section 7 for SO (2020) to section_7_mda_results/SO_2020_section_7_mda.csv.
Saved Section 7 for SO (2018) to section_7_mda_results/SO_2018_section_7_mda.csv.
No 'Item 7' or variations found in sec-edgar-filings/SO/10-K/0000092122-02-000045/full-submission.txt.
No 'Item 7' or

In [ ]:
import os
import pandas as pd
import re

def sort_data_year_wise_combined(input_folder, output_folder, start_year, end_year):
    # Create the output folder for year-wise data
    os.makedirs(output_folder, exist_ok=True)

    # Initialize a dictionary to hold data for each year
    year_data = {str(year): {} for year in range(start_year, end_year + 1)}

    # Process each CSV file in the input folder
    for file in os.listdir(input_folder):
        if file.endswith(".csv"):
            file_path = os.path.join(input_folder, file)
            try:
                # Read the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Extract the ticker and year from the filename
                match = re.search(r'([^_]+)_(\d{4})_section_7_mda\.csv', file)
                if match:
                    ticker = match.group(1)
                    year = match.group(2)

                    if year in year_data:
                        # Combine all section titles and text into a single string for the company
                        combined_text = " ".join(
                            f"{row['Section Title']}: {row['Content']}" for _, row in df.iterrows()
                        )

                        # Add or update the company's combined MD&A text for the year
                        if ticker in year_data[year]:
                            year_data[year][ticker] += " " + combined_text
                        else:
                            year_data[year][ticker] = combined_text
                else:
                    print(f"Invalid file format: {file}")

            except Exception as e:
                print(f"Error processing file {file}: {e}")

    # Save each year's data to a separate CSV
    for year, companies in year_data.items():
        if companies:  # Only process years with data
            year_df = pd.DataFrame(list(companies.items()), columns=['Ticker', 'Combined MD&A'])
            year_file = os.path.join(output_folder, f"{year}.csv")
            year_df.to_csv(year_file, index=False, encoding="utf-8")
            print(f"Saved data for year {year} to {year_file}.")

# Example usage
if __name__ == "__main__":
    # Directory containing the extracted Section 7 data
    input_folder = "section_7_mda_results"

    # Directory to save year-wise data
    output_folder = "yearwise_data"

    # Specify the range of years
    start_year = 1995
    end_year = 2024

    # Sort and save data year-wise
    sort_data_year_wise_combined(input_folder, output_folder, start_year, end_year)


Saved data for year 2002 to yearwise_data/2002.csv.
Saved data for year 2003 to yearwise_data/2003.csv.
Saved data for year 2004 to yearwise_data/2004.csv.
Saved data for year 2005 to yearwise_data/2005.csv.
Saved data for year 2006 to yearwise_data/2006.csv.
Saved data for year 2007 to yearwise_data/2007.csv.
Saved data for year 2008 to yearwise_data/2008.csv.
Saved data for year 2009 to yearwise_data/2009.csv.
Saved data for year 2010 to yearwise_data/2010.csv.
Saved data for year 2011 to yearwise_data/2011.csv.
Saved data for year 2012 to yearwise_data/2012.csv.
Saved data for year 2013 to yearwise_data/2013.csv.
Saved data for year 2014 to yearwise_data/2014.csv.
Saved data for year 2015 to yearwise_data/2015.csv.
Saved data for year 2016 to yearwise_data/2016.csv.
Saved data for year 2017 to yearwise_data/2017.csv.
Saved data for year 2018 to yearwise_data/2018.csv.
Saved data for year 2019 to yearwise_data/2019.csv.
Saved data for year 2020 to yearwise_data/2020.csv.
Saved data f

In [ ]:
import shutil
from google.colab import files

# Path to the folder you want to download
folder_to_download = "yearwise_data"

# Create a ZIP file of the folder
shutil.make_archive(folder_to_download, 'zip', folder_to_download)

# Download the ZIP file to your local device
files.download(f"{folder_to_download}.zip")


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

def plot_companies_per_year(output_folder):
    # Dictionary to store year and count of companies
    year_company_counts = {}

    # Loop through each year's CSV file
    for file in os.listdir(output_folder):
        if file.endswith(".csv"):
            year = os.path.splitext(file)[0]  # Extract the year from the filename
            file_path = os.path.join(output_folder, file)
            try:
                # Read the CSV file
                df = pd.read_csv(file_path)

                # Count the number of rows (companies) in the file
                year_company_counts[year] = len(df)
            except Exception as e:
                print(f"Error reading file {file}: {e}")

    # Sort the dictionary by year (convert keys to integers for sorting)
    sorted_years = sorted(year_company_counts.keys(), key=lambda x: int(x))
    sorted_counts = [year_company_counts[year] for year in sorted_years]

    # Plot the bar chart
    plt.figure(figsize=(12, 6))
    plt.bar(sorted_years, sorted_counts, color='skyblue')
    plt.xlabel('Year')
    plt.ylabel('Number of Companies')
    plt.title('Number of Companies Per Year')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Example usage
if __name__ == "__main__":
    output_folder = "yearwise_data"
    plot_companies_per_year(output_folder)


In [ ]:
df=pd.read_csv("/content/yearwise_data/2023.csv")

In [ ]:
df['Combined MD&A'][0]

'ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS.: ITEM\xa07. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS. (In Millions, Except Per Share Data or As Otherwise Stated Herein)On May 18, 2022, our shareholders approved a proposal to amend our amended and restated articles of incorporation to change our name from Anthem, Inc. to Elevance Health, Inc. This amendment and name change went into effect on June 27, 2022. We began operating as Elevance Health, Inc. and trading under our new ticker symbol “ELV” on June 28, 2022. References to the terms “we,” “our,” “us,” “Elevance Health” or the “Company” used throughout this Management’s Discussion and Analysis of Financial Condition and Results of Operations (“MD&A”) refer to Elevance Health, Inc., an Indiana corporation, and, unless the context otherwise requires, its direct and indirect subsidiaries. References to the “states” include the District of Columb